In [1]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostRanker,Pool

In [2]:
train=pd.read_parquet('../Data/Big_train_feature_engineered_2.parquet',engine='pyarrow')
val=pd.read_parquet('../Data/Big_val_feature_engineered_2.parquet',engine='pyarrow')

In [3]:
train

,id2,id3,id4,id5,y,f1,f2,f5,f6,f7,...,num_offer_categories,sub_category,num_sub_categories,previous_offer_category,previous_suboffer_category,no_of_clicks_previously,pace_x_ctr,session_count_x_ctr,pace_x_offer_popularity,time_since_last_click
228640,1000078,622070,2023-11-02 03:40:44.605,2023-11-02,0,NaN,NaN,NaN,NaN,NaN,...,1,Computer services,1,missing,missing,0,NaN,NaN,NaN,NaN
46930,1000101,69479,2023-11-02 12:54:11.194,2023-11-02,0,NaN,NaN,NaN,NaN,NaN,...,1,Other,1,missing,missing,0,NaN,0.023810,NaN,NaN
46929,1000101,86651,2023-11-02 12:54:46.722,2023-11-02,0,NaN,NaN,NaN,NaN,NaN,...,1,Other,1,Shopping,Other,0,0.845905,0.047619,NaN,NaN
12738,1000125,78680,2023-11-01 21:06:27.987,2023-11-01,0,NaN,26.0,22.0,NaN,NaN,...,1,Clothing,1,missing,missing,0,NaN,0.004184,NaN,NaN
12692,1000125,481197,2023-11-01 21:06:28.369,2023-11-01,0,NaN,26.0,22.0,NaN,NaN,...,1,Other,1,Shopping,Clothing,0,0.001598,0.008368,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456767,1910541,5420674,2023-11-01 07:43:31.173,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,1,Other,1,Shopping,Other,0,0.000000,0.000000,NaN,NaN
456765,1910541,82192,2023-11-01 07:43:31.452,2023-11-01,0,NaN,NaN,63.0,65.0,NaN,...,1,Clothing,1,Shopping,Other,0,0.000000,0.000000,0.021704,NaN
456769,1910541,285330,2023-11-02 06:37:12.625,2023-11-02,0,NaN,NaN,63.0,65.0,NaN,...,1,None,0,Shopping,Clothing,0,0.000000,0.000000,14364.764227,NaN
63239,1910565,415582,2023-11-01 11:12:58.728,2023-11-01,0,NaN,NaN,24.0,13.0,19.0,...,1,Auto services,1,missing,missing,0,NaN,NaN,NaN,NaN


In [4]:
# --- YOUR HACKATHON METRIC FUNCTION ---
def map_at_k_hackathon(y_true, y_pred_scores, group_ids, k=7):
    """
    Calculates the Mean Average Precision at k.
    """
    df = pd.DataFrame({'group': group_ids, 'y_true': y_true, 'score': y_pred_scores})
    
    average_precisions = []
    total_relevant_per_group = df[df['y_true'] == 1].groupby('group')['y_true'].count()

    for group_id, group_df in df.groupby('group'):
        total_relevant = total_relevant_per_group.get(group_id, 0)
        if total_relevant == 0:
            continue

        group_df = group_df.sort_values('score', ascending=False).head(k)
        
        hits = 0
        precision_sum = 0.0
        
        for i, row in enumerate(group_df.itertuples(index=False)):
            rank = i + 1
            if row.y_true == 1:
                hits += 1
                precision_at_k = hits / rank
                precision_sum += precision_at_k
        
        ap = precision_sum / total_relevant
        average_precisions.append(ap)

    return np.mean(average_precisions) if average_precisions else 0.0


In [5]:
cat_features_names = train.select_dtypes(include=['category']).columns.tolist()
cat_features_names.remove('id2')
cat_features_names.remove('id3')

In [6]:
for df in [train, val]:
    for col in cat_features_names:
        if df[col].dtype.name == 'category':
            df[col] = df[col].cat.add_categories('NotAvailable')
    df[cat_features_names] = df[cat_features_names].fillna('NotAvailable')
print("Successfully handled null values.")

Successfully handled null values.


In [7]:
# --- 1. Preprocessing & Data Preparation ---
# (Assuming preprocessing like handling nulls is already done)
TARGET = 'y'
FEATURES = [col for col in train.columns if col not in [TARGET, 'id2', 'id3','id4','id5']]

In [8]:

# CRITICAL: Sort data by the user ID
train = train.sort_values(['id2','id4']).reset_index(drop=True)
val = val.sort_values(['id2','id4']).reset_index(drop=True)


In [9]:

# Define your data sets
X_train = train[FEATURES]

In [10]:

y_train = train[TARGET].astype(int)


In [11]:
X_val = val[FEATURES]


In [12]:

y_val = val[TARGET].astype(int)


In [13]:

# Identify categorical features
cat_features_names = X_train.select_dtypes(include=['category', 'object']).columns.tolist()
cat_features_indices = [X_train.columns.get_loc(col) for col in cat_features_names]


In [14]:


# --- 2. Create catboost.Pool Objects ---
# Bundle the training data into a Pool
train_pool = Pool(
    data=X_train,
    label=y_train,
    group_id=train['id2'],
    cat_features=cat_features_indices
)


In [15]:

# Bundle the validation data into a Pool
validation_pool = Pool(
    data=X_val,
    label=y_val,
    group_id=val['id2'],
    cat_features=cat_features_indices
)


In [16]:


# --- 3. Model Training ---
model = CatBoostRanker(
    iterations=1500,
    learning_rate=0.05,
    random_seed=42,
    verbose=100,
    loss_function='LambdaMart',
    eval_metric='MAP:top=7'
)


In [17]:

print("\nStarting CatBoostRanker training...")

# Pass the Pool objects to the .fit() method
model.fit(
    train_pool,
    eval_set=validation_pool,
    early_stopping_rounds=100,
    use_best_model=True
)



Starting CatBoostRanker training...
Groupwise loss function. OneHotMaxSize set to 10
0:	learn: 0.0499669	test: 0.0440113	best: 0.0440113 (0)	total: 413ms	remaining: 10m 18s
100:	learn: 0.0706447	test: 0.0620923	best: 0.0620923 (100)	total: 27.6s	remaining: 6m 21s
200:	learn: 0.0749901	test: 0.0654067	best: 0.0654659 (195)	total: 53.7s	remaining: 5m 47s
300:	learn: 0.0774780	test: 0.0675165	best: 0.0675165 (300)	total: 1m 20s	remaining: 5m 20s
400:	learn: 0.0791674	test: 0.0683914	best: 0.0684293 (395)	total: 1m 46s	remaining: 4m 52s
500:	learn: 0.0805794	test: 0.0689595	best: 0.0689595 (500)	total: 2m 13s	remaining: 4m 26s
600:	learn: 0.0818174	test: 0.0694626	best: 0.0694667 (599)	total: 2m 44s	remaining: 4m 6s
700:	learn: 0.0828029	test: 0.0694363	best: 0.0696008 (672)	total: 3m 14s	remaining: 3m 41s
800:	learn: 0.0838577	test: 0.0697444	best: 0.0698203 (791)	total: 3m 43s	remaining: 3m 14s
900:	learn: 0.0847341	test: 0.0697968	best: 0.0699743 (868)	total: 4m 11s	remaining: 2m 46s
1

In [18]:

# --- 4. Prediction ---
# Prediction still works on the DataFrame directly
val_scores = model.predict(X_val)

print("\nTraining complete!")
print(f"Best validation MAP@7: {model.get_best_score()['validation']['MAP:top=7']:.4f}")


Training complete!
Best validation MAP@7: 0.0702


In [19]:
map7 = map_at_k_hackathon(y_val.values, val_scores, val['id2'].values, k=7)
print(f"Hackathon MAP@7: {map7:.4f}")

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_13682/3676040123.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_relevant_per_group = df[df['y_true'] == 1].groupby('group')['y_true'].count()
/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_13682/3676040123.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for group_id, group_df in df.groupby('group'):


Hackathon MAP@7: 0.6533
